In [ ]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PolicyData

/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PolicyData


In [ ]:
import os
import sys
import re
import pickle
import csv
import ast
import numpy as np
import pandas as pd
from pandas import DataFrame 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 500)

In [ ]:
data = pd.read_csv('정책_preprocessed.csv', encoding = 'utf-8')

In [ ]:
data.head()

,분야,번호,정책명,정책내용,year,period,nouns
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,법정 최고금리 연 로 인하 법무부 상사법무과 금...,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축 법무부 상사법무과 분야 금...,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입 방송통신위원회 개인정보보호윤리과 ...,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,외국인관광객 숙박요금 부가가치세 환급 문화체육관광부 관광산업정책과 ...,2018,상반기,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점 조달청 기술서비스총괄과 ...,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...


In [ ]:
data.groupby(['year','period']).count()['번호']

year  period
2018  상반기        83
      하반기        30
2019  상반기       126
      하반기       178
2020  상반기       279
      하반기       153
2021  상반기       296
      하반기       166
Name: 번호, dtype: int64

In [ ]:
data.groupby(['year','period']).nunique()['분야']

year  period
2018  상반기        2
      하반기        2
2019  상반기        3
      하반기       10
2020  상반기       10
      하반기       10
2021  상반기       10
      하반기       10
Name: 분야, dtype: int64

In [ ]:
data.groupby(['year','period','분야']).count()['번호']

year  period  분야         
2018  상반기     국방/병무          20
              금융/재정/조세       63
      하반기     국방/병무          20
              금융/재정/조세       10
2019  상반기     국방/병무          20
              금융/재정/조세       99
              문화/체육/관광        7
      하반기     교육/보육/가족        8
              국방/병무          13
              국토/교통          11
              금융/재정/조세       25
              농림/수산/식품       25
              문화/체육/관광        5
              보건/복지/고용       35
              산업/중소기업/에너지    16
              행정/안전/질서       25
              환경/기상          15
2020  상반기     교육/보육/가족       20
              국방/병무          14
              국토/교통           6
              금융/재정/조세       66
              농림/수산/식품       32
              문화/체육/관광       12
              보건/복지/고용       40
              산업/중소기업/에너지    18
              행정/안전/질서       46
              환경/기상          25
      하반기     교육/보육/가족        2
              국방/병무          15
              국토/교통           2
              

In [ ]:
years = [2018,2019,2020,2021]
halfs = ['상반기','하반기']
#fields = list(data['분야'].unique())

# TF-IDF 키워드 추출

In [ ]:
from collections import Counter

In [ ]:
ydf = []
hdf = []
fdf = []
corpus = []
forstopw = []
for yy in years:
    for hh in halfs:
        #for ff in fields:
        temp = data[(data['year']==yy) & (data['period']==hh)]#& (data['분야']==ff)
        if len(temp)==0:
            continue
        ydf.append(yy)
        hdf.append(hh)
        #fdf.append(ff)
        tmp = temp['nouns'].to_list()
        for i in tmp:
            forstopw.extend(i.split())
        corpus.append(' '.join(tmp))


In [ ]:
corpus

['법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상 사회 약자 취약 계층 관련 부처 법무부 서민 금융 부담 경감 위해 법정 최고 금리 모두 인하 예정 지금 법정 최고 금리 일반 사인 금전 거래 경우 대부 업자 금융기관 경우 사인 일반 금전 거래 대부 업자 금융기관 법정 최고 금리 모두 계획 인하 법정 최고 금리 신규 체결 갱신 연장 계약 적용 참고 사이트 법무부 홈페이지 법무 뉴스 보도자료 이자제한법 시행 정령 국무회의 통과 금융 위원회 홈페이지 알림 마당 보도자료 부터 법정 최고 금리 인하 첨부파일 법무부 법정 최고 금리 인하 크기 개정 내용 법정 최고 금리 인하 일원화 추진 배경 경제 취약 계층 금리 부담 경감 가계부채 해소 대책 마련 내용 법정 최고 금리 인하 이자제한법 부업 최고 금리 일원화 이자제한법 부업 시행 개정 최고 금리 인하 공포 체결 갱신 연장 계약 적용 시행일 전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 기업 소상 공인 관련 부처 법무부 장기 어음 폐해 해소 중소기업 상공 부담 경감 위해 전자 어음 만기 단계 단축 예정 현재 전자 어음 만기 발행 어음 제도 폐해 해소 위해 만기 단축 내용 개정 전자 어음 발행 유통 법률 공포 시행 예정 개월 초과 우선 시행 매년 개월 단축 개월 초과 단축 계획 만기 단축 해당 시행 시기 이후 발행 적용 참고 사이트 법무부 홈페이지 법무 뉴스 보도자료 어음 만기 단축 경제 활성화 기업 환경 개선 첨부파일 법무부 전자 어음 만기 단계 단축 크기 개정 내용 어음 만기 단축 전자 어음 개정 추진 배경 장기 만기 어음 어음 폐해 해소 기업 환경 개선 내용 전자 어음 만기 단계 단축 공포 개월 단축 이후 매년 개월 단축 최종 공포 개월 단축 시행일 신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야 금융 재정 조세 대상 일반 국민 관련 부처 방송통신 위원회 상반기 신용카드 활용 신규 본인 확인 서비스 시작 지금 아

In [ ]:
forstopw

['법정',
 '최고',
 '금리',
 '인하',
 '법무부',
 '상사',
 '법무',
 '금융위',
 '서민',
 '금융',
 '분야',
 '금융',
 '재정',
 '조세',
 '대상',
 '사회',
 '약자',
 '취약',
 '계층',
 '관련',
 '부처',
 '법무부',
 '서민',
 '금융',
 '부담',
 '경감',
 '위해',
 '법정',
 '최고',
 '금리',
 '모두',
 '인하',
 '예정',
 '지금',
 '법정',
 '최고',
 '금리',
 '일반',
 '사인',
 '금전',
 '거래',
 '경우',
 '대부',
 '업자',
 '금융기관',
 '경우',
 '사인',
 '일반',
 '금전',
 '거래',
 '대부',
 '업자',
 '금융기관',
 '법정',
 '최고',
 '금리',
 '모두',
 '계획',
 '인하',
 '법정',
 '최고',
 '금리',
 '신규',
 '체결',
 '갱신',
 '연장',
 '계약',
 '적용',
 '참고',
 '사이트',
 '법무부',
 '홈페이지',
 '법무',
 '뉴스',
 '보도자료',
 '이자제한법',
 '시행',
 '정령',
 '국무회의',
 '통과',
 '금융',
 '위원회',
 '홈페이지',
 '알림',
 '마당',
 '보도자료',
 '부터',
 '법정',
 '최고',
 '금리',
 '인하',
 '첨부파일',
 '법무부',
 '법정',
 '최고',
 '금리',
 '인하',
 '크기',
 '개정',
 '내용',
 '법정',
 '최고',
 '금리',
 '인하',
 '일원화',
 '추진',
 '배경',
 '경제',
 '취약',
 '계층',
 '금리',
 '부담',
 '경감',
 '가계부채',
 '해소',
 '대책',
 '마련',
 '내용',
 '법정',
 '최고',
 '금리',
 '인하',
 '이자제한법',
 '부업',
 '최고',
 '금리',
 '일원화',
 '이자제한법',
 '부업',
 '시행',
 '개정',
 '최고',
 '금리',
 '인하',
 '공포',
 '체결',
 '갱신',
 '연장'

In [ ]:
len(Counter(forstopw))

7396

In [ ]:
74/7396

0.010005408328826392

In [ ]:
stopwords = [x for x,y in Counter(forstopw).most_common(80)]

In [ ]:
Counter(forstopw).most_common(80)

[('내용', 2597),
 ('대상', 2477),
 ('개정', 1970),
 ('지원', 1762),
 ('관련', 1731),
 ('분야', 1530),
 ('추진', 1529),
 ('확대', 1428),
 ('적용', 1411),
 ('부처', 1363),
 ('시행일', 1282),
 ('배경', 1235),
 ('경우', 1119),
 ('시행', 1013),
 ('사업', 1012),
 ('기타', 999),
 ('관리', 960),
 ('대한', 913),
 ('기준', 870),
 ('제도', 822),
 ('안전', 755),
 ('기업', 735),
 ('서비스', 724),
 ('정보', 716),
 ('기획재정부', 696),
 ('참고', 684),
 ('개선', 658),
 ('위해', 648),
 ('금융', 646),
 ('홈페이지', 645),
 ('보도자료', 628),
 ('산업', 604),
 ('이상', 601),
 ('기간', 571),
 ('지역', 565),
 ('제공', 563),
 ('신고', 563),
 ('의무', 561),
 ('신청', 560),
 ('운영', 558),
 ('시설', 550),
 ('정책', 541),
 ('교육', 540),
 ('강화', 533),
 ('사이트', 532),
 ('예정', 512),
 ('기술', 497),
 ('만원', 493),
 ('고용', 467),
 ('현행', 467),
 ('기관', 466),
 ('지급', 435),
 ('보호', 431),
 ('국민', 429),
 ('이후', 422),
 ('국가', 422),
 ('등록', 419),
 ('계획', 414),
 ('소득', 413),
 ('환경', 399),
 ('통해', 395),
 ('보험', 394),
 ('이용', 384),
 ('신설', 369),
 ('부담', 367),
 ('추가', 367),
 ('재정', 366),
 ('복지', 366),
 ('도입', 363),
 ('평가', 3

In [ ]:
ttt = []
for i in forstopw:
  if i.endswith('부') and len(i)>3:
    ttt.append(i)

In [ ]:
list(set(ttt))

['열대저압부',
 '정보통신부',
 '소방본부',
 '농식품부',
 '임시정부',
 '여성가족부',
 '농림축산식품부',
 '해군본부',
 '교정본부',
 '기획재정부',
 '행정안전부',
 '육군본부',
 '국토교통부',
 '전자정부',
 '공군본부',
 '신혼부부',
 '보건복지부',
 '해양수산부',
 '고용노동부']

In [ ]:
ttt.remove('신혼부부')

In [ ]:
stopwords.extend(ttt)

In [ ]:
header = {'연도':ydf,'분기':hdf}#,'분야':fdf}
header = pd.DataFrame(header)

In [ ]:
vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True, stop_words= stopwords)

In [ ]:
vectorizer.fit_transform(corpus)

<8x7298 sparse matrix of type '<class 'numpy.float64'>'
	with 19337 stored elements in Compressed Sparse Row format>

In [ ]:
feature_names = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [ ]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
TOP_K_KEYWORDS = 20

In [ ]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())

In [ ]:
header

,연도,분기
0,2018,상반기
1,2018,하반기
2,2019,상반기
3,2019,하반기
4,2020,상반기
5,2020,하반기
6,2021,상반기
7,2021,하반기


In [ ]:
header.iloc[0].연도

2018

In [ ]:
data.head()

,분야,번호,정책명,정책내용,year,period,nouns
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,법정 최고금리 연 로 인하 법무부 상사법무과 금...,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축 법무부 상사법무과 분야 금...,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입 방송통신위원회 개인정보보호윤리과 ...,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,외국인관광객 숙박요금 부가가치세 환급 문화체육관광부 관광산업정책과 ...,2018,상반기,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점 조달청 기술서비스총괄과 ...,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...


In [ ]:
data['score'] = 0

In [ ]:
data.head()

,분야,번호,정책명,정책내용,year,period,nouns,score
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,법정 최고금리 연 로 인하 법무부 상사법무과 금...,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...,0
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축 법무부 상사법무과 분야 금...,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...,0
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입 방송통신위원회 개인정보보호윤리과 ...,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...,0
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,외국인관광객 숙박요금 부가가치세 환급 문화체육관광부 관광산업정책과 ...,2018,상반기,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...,0
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점 조달청 기술서비스총괄과 ...,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...,0


In [ ]:
data.iloc[3].nouns.split()

['외국인',
 '관광객',
 '숙박',
 '요금',
 '부가가치세',
 '환급',
 '문화',
 '체육관',
 '광부',
 '관광',
 '산업',
 '정책',
 '분야',
 '금융',
 '재정',
 '조세',
 '대상',
 '영유',
 '아동',
 '관련',
 '부처',
 '문화',
 '체육관',
 '광부',
 '외국인',
 '관광객',
 '문화',
 '체육관',
 '광부',
 '지정',
 '관광',
 '호텔',
 '이하',
 '숙박',
 '경우',
 '숙박',
 '요금',
 '대한',
 '부가가치세',
 '환급',
 '계획',
 '한시',
 '시행',
 '숙박',
 '요금',
 '인하',
 '효과',
 '외국인',
 '관광객',
 '증대',
 '위해',
 '실시',
 '예정',
 '환급',
 '대상',
 '호텔',
 '지정',
 '문화',
 '체육관',
 '광부',
 '신청',
 '숙박',
 '이용',
 '외국인',
 '관광객',
 '환급',
 '전표',
 '발급',
 '환급',
 '창구',
 '환급',
 '실시',
 '환급',
 '대상',
 '확대',
 '이상',
 '조건',
 '이상',
 '구역',
 '도심',
 '환급',
 '창구',
 '제도',
 '보완',
 '참고',
 '사이트',
 '문화',
 '체육관',
 '광부',
 '고시',
 '외국인',
 '관광객',
 '숙박',
 '용역',
 '대한',
 '부가가치세',
 '특례',
 '관련',
 '고시',
 '개정',
 '내용',
 '외국인',
 '관광객',
 '숙박',
 '요금',
 '부가가치세',
 '환급',
 '추진',
 '배경',
 '숙박',
 '요금',
 '인하',
 '효과',
 '외국인',
 '관광객',
 '증대',
 '내용',
 '외국인',
 '관광객',
 '문화',
 '체육관',
 '광부',
 '지정',
 '관광',
 '호텔',
 '이하',
 '숙박',
 '경우',
 '숙박',
 '요금',
 '대한',
 '부가가치세',
 '환급',
 '시행일']

In [ ]:
data.head()

,분야,번호,정책명,정책내용,year,period,nouns,score
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,법정 최고금리 연 로 인하 법무부 상사법무과 금...,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...,0
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축 법무부 상사법무과 분야 금...,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...,0
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입 방송통신위원회 개인정보보호윤리과 ...,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...,0
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,외국인관광객 숙박요금 부가가치세 환급 문화체육관광부 관광산업정책과 ...,2018,상반기,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...,0
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점 조달청 기술서비스총괄과 ...,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...,0


In [ ]:
result = []
policys = []
scores = [0 for i in range(len(data))]
for i in range(len(corpus)):
    doc = corpus[i]
    kws = get_keywords(vectorizer, feature_names, doc)
    yy, hh = header.iloc[i].연도, header.iloc[i].분기
    temp = data[(data['year']==yy) & (data['period']==hh)]
    for pp in temp['nouns'].index:
        wds = data.iloc[pp].nouns.split()
        scr = 0
        for w in wds:
            if w in kws:
                scr+=1
        data.loc[pp,'score'] = scr
    df = {}
    df['full_text'] = doc
    df['top_keywords'] = kws

    result.append(df)

final = pd.DataFrame(result)
final

,full_text,top_keywords
0,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...,"[관세, 양도소득세, 수입, 여군, 한도, 일반, 기한, 주식, 세액, 납세의무, ..."
1,주거 급여 제도 부양 의무 기준 폐지 국토교통부 주거 복지 정책 분야 금융 재정 조...,"[국방, 입영, 연기, 함정, 군무원, 선발, 연평해전, 근조기, 귀화, 유공, 경..."
2,중소 중견 기업 소상 공인 자영 업자 금융 부담 완화 금융 위원회 산업 금융 분야 ...,"[세액, 기한, 연장, 일반, 세법개정안, 관세, 납부, 부가가치세, 면세점, 금액..."
3,개인 정보보호 손해배상 책임 보험 가입 의무 방송통신 위원회 개인 정보보호 윤리 분...,"[기상, 첨부파일, 법령, 지정, 검사, 건강, 업체, 장애인, 표시, 의료기기, ..."
4,억원 주택 취득 세율 세분 주택 적용 행정안전부 부동산 제과 분야 금융 재정 조세 ...,"[청소년, 환경부, 인증, 배출, 문화, 창업, 제품, 납부, 식품, 부과, 센터,..."
5,보이스피싱 피해 예방 강화 금융 위원회 전자 금융 분야 금융 재정 조세 대상 기타 ...,"[문화재, 어린이, 청소년, 질서, 계약, 규정, 행정, 기상, 발생, 아동, 검사..."
6,통합 투자 세액 공제 신설 기획재정부 특례 제도 분야 금융 재정 조세 대상 기타 관...,"[수산, 식품, 개발, 해양, 보건, 의료, 주택, 구축, 코로나, 검사, 행정, ..."
7,지급 명세서 제출 주기 단축 고용 보험 소득 정보 추진단 분야 금융 재정 조세 대상...,"[행정, 코로나, 센터, 대응, 복무, 규정, 확인, 법률, 질서, 예방, 인증, ..."


In [ ]:
fin = pd.concat([header, final], axis=1)

In [ ]:
fin

,연도,분기,full_text,top_keywords
0,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...,"[관세, 양도소득세, 수입, 여군, 한도, 일반, 기한, 주식, 세액, 납세의무, ..."
1,2018,하반기,주거 급여 제도 부양 의무 기준 폐지 국토교통부 주거 복지 정책 분야 금융 재정 조...,"[국방, 입영, 연기, 함정, 군무원, 선발, 연평해전, 근조기, 귀화, 유공, 경..."
2,2019,상반기,중소 중견 기업 소상 공인 자영 업자 금융 부담 완화 금융 위원회 산업 금융 분야 ...,"[세액, 기한, 연장, 일반, 세법개정안, 관세, 납부, 부가가치세, 면세점, 금액..."
3,2019,하반기,개인 정보보호 손해배상 책임 보험 가입 의무 방송통신 위원회 개인 정보보호 윤리 분...,"[기상, 첨부파일, 법령, 지정, 검사, 건강, 업체, 장애인, 표시, 의료기기, ..."
4,2020,상반기,억원 주택 취득 세율 세분 주택 적용 행정안전부 부동산 제과 분야 금융 재정 조세 ...,"[청소년, 환경부, 인증, 배출, 문화, 창업, 제품, 납부, 식품, 부과, 센터,..."
5,2020,하반기,보이스피싱 피해 예방 강화 금융 위원회 전자 금융 분야 금융 재정 조세 대상 기타 ...,"[문화재, 어린이, 청소년, 질서, 계약, 규정, 행정, 기상, 발생, 아동, 검사..."
6,2021,상반기,통합 투자 세액 공제 신설 기획재정부 특례 제도 분야 금융 재정 조세 대상 기타 관...,"[수산, 식품, 개발, 해양, 보건, 의료, 주택, 구축, 코로나, 검사, 행정, ..."
7,2021,하반기,지급 명세서 제출 주기 단축 고용 보험 소득 정보 추진단 분야 금융 재정 조세 대상...,"[행정, 코로나, 센터, 대응, 복무, 규정, 확인, 법률, 질서, 예방, 인증, ..."


In [ ]:
' '.join(fin.iloc[4].top_keywords)

'청소년 환경부 인증 배출 문화 창업 제품 납부 식품 부과 센터 지정 인상 장애인 의료 검사 친환경 세액 질서 법령'

In [ ]:
data[data['score']!=0]

,분야,번호,정책명,정책내용,year,period,nouns,score
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,법정 최고금리 연 로 인하 법무부 상사법무과 금...,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...,3
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축 법무부 상사법무과 분야 금...,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...,1
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입 방송통신위원회 개인정보보호윤리과 ...,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...,2
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점 조달청 기술서비스총괄과 ...,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...,5
5,금융/재정/조세,6,조달청 물품구매 적격심사 및 계약이행능력 심사 시 사회적책임 강화,조달청 물품구매 적격심사 및 계약이행능력 심사 시 사회적책임 강화 조달청 구매총괄과...,2018,상반기,조달청 물품 구매 격심 계약 이행 능력 심사 사회 책임 강화 조달청 총괄 분야 금융...,9
...,...,...,...,...,...,...,...,...
1306,국토/교통,2,국내공항 짐배송서비스 시범사업 추진,국내공항 짐배송서비스 시범사업 추진 항공보안과 분야 국토...,2021,하반기,국내 공항 배송 서비스 시범 사업 추진 항공 보안 분야 국토 교통 대상 기타 관련 ...,2
1307,국토/교통,3,수도권에도 드론전용비행시험장이 운영됩니다,수도권에도 드론전용비행시험장이 운영됩니다 첨단항공과 분야...,2021,하반기,수도권 드론 전용 비행 시험 운영 첨단 항공 분야 국토 교통 대상 기타 관련 부처 ...,1
1308,국토/교통,4,항공 안전데이터 분석센터의 운영 시작,항공 안전데이터 분석센터의 운영 시작 항공안전정책과 분야...,2021,하반기,항공 안전 데이터 분석 센터 운영 시작 항공 안전 정책 분야 국토 교통 대상 기타 ...,9
1309,국토/교통,5,소화물배송대행서비스 사업자 인증제 시행,소화물배송대행서비스 사업자 인증제 시행 생활물류 긴급대응반 상황총괄대응과 ...,2021,하반기,화물 배송 대행 서비스 사업자 인증 시행 생활 물류 긴급 대응 상황 총괄 대응 분야...,11


In [ ]:
data.dtypes

분야        object
번호         int64
정책명       object
정책내용      object
year       int64
period    object
nouns     object
score      int64
dtype: object

In [ ]:

fig = px.box(data, y = 'score')
fig.show()

In [ ]:
pc31 = [1216,1194,1207,1301,1211,1208,1295,1298]
pc30 = [1126,946,1111,910,907]
pc21 = [707,742,818,749, 735, 734, 738, 737,752,830,743]
pc20 = [490,544,545,481,546]

In [ ]:
data.iloc[pc31]

,분야,번호,정책명,정책내용,year,period,nouns,score
1216,행정/안전/질서,25,기술보유 혁신 중소기업의 입찰등록 간소화,기술보유 혁신 중소기업의 입찰등록 간소화 조달등록팀 분야...,2021,하반기,기술 보유 혁신 중소기업 입찰 등록 간소화 조달 등록 분야 행정 안전 질서 대상 기...,18
1194,행정/안전/질서,3,법무부 코로나19 교정시설 긴급대응단 운영 실시,법무부 코로나 교정시설 긴급대응단 운영 실시 코로나 긴급대응단 ...,2021,하반기,법무부 코로나 교정 시설 긴급 대응 운영 실시 코로나 긴급 대응 분야 행정 안전 질...,30
1207,행정/안전/질서,16,"온라인에서의 아동·청소년 성적 착취 목적의 그루밍 처벌, 아동·청소년 대상 디지털 ...",온라인에서의 아동 청소년 성적 착취 목적의 그루밍 처벌 아동 청소년 대상 디지털 ...,2021,하반기,온라인 아동 청소년 성적 착취 목적 그루밍 처벌 아동 청소년 대상 디지털 성범죄 분...,24
1301,보건/복지/고용,24,코로나19 백신 개발을 위한 맞춤형 전주기 품질관리 및 신속 국가출하승인을 통한 집...,코로나 백신 개발을 위한 맞춤형 전주기 품질관리 및 신속 국가출하승인을 통한 집...,2021,하반기,코로나 백신 개발 맞춤 주기 품질 관리 신속 국가 출하 승인 통한 집단 면역 가속 ...,15
1211,행정/안전/질서,20,디지털 성범죄 피해영상물 삭제 지원 강화,디지털 성범죄 피해영상물 삭제 지원 강화 권익침해방지과 ...,2021,하반기,디지털 성범죄 피해 영상 삭제 지원 강화 권익 침해 방지 분야 행정 안전 질서 대상...,13
1208,행정/안전/질서,17,성범죄자 신상정보 모바일 고지 확대,성범죄자 신상정보 모바일 고지 확대 아동청소년성보호과 분...,2021,하반기,성범죄자 신상 정보 모바일 고지 확대 아동 청소년 보호 분야 행정 안전 질서 대상 ...,10
1295,보건/복지/고용,18,「공정안전보고서 제출 대상 유해·위험물질 규정량 조정」 5인 미만 사업장 시행,공정안전보고서 제출 대상 유해 위험물질 규정량 조정 인 미만 사업장 시행 화학...,2021,하반기,공정 안전 보고서 제출 대상 유해 위험 물질 정량 조정 미만 사업 시행 화학사 예방...,9
1298,보건/복지/고용,21,산재보험 진료비 본인부담금 확인제도 시행,산재보험 진료비 본인부담금 확인제도 시행 산재보상정책과 ...,2021,하반기,산재 보험 진료 인부 확인 제도 시행 산재 보상 정책 분야 보건 복지 고용 대상 기...,20


In [ ]:
data.iloc[1207].정책명

'온라인에서의 아동·청소년 성적 착취 목적의 그루밍 처벌, 아동·청소년 대상 디지털 성범죄 신분위장수사로 적발'

In [ ]:

'''
3-1 : 
1216 : 2020년 하반기 9번 - 중소기업 : 기술 보유 혁싱 중소기업 등록관련 = 항상 등장하는 기업 기술 관련 토픽 (20년도 하반기 9번)
1194 : 교정시설 코로나(2020년 하반기 구치소, 4,7,10) (21년 동부 구치소 7)
1207 : 아동 성 착취 (21년 상반기 아동학대 7번토픽)
1301 : 백신 개발 관련 정책
1211 : 디지털 성범죄 (19년 상반기 5번)
1208 : 성범죄자 신상정보 모바일 고지 확대	
1295 : 산업재해 「공정안전보고서 제출 대상 유해·위험물질 규정량 조정」 5인 미만 사업장 시행	#2018하반기
1298 : 산재보험 #2018하반기


3-0:
1126 : 코로나 19 대응 제품 개발 관련 
946	: post코로나 대비
1111 : 산업재해 「물질안전보건자료(MSDS) 제출 및 대체자료 기재 심사제도」 시행
910, 907:  교육 : 보조·연장보육교사 확대 배치	
####880, 851: 양도소득세


#2-1: 주로 어린이 위주
707,742, :  성매매 대상아동·청소년의 피해자화 등 아동·청소년 성보호 정책 기반 강화, 아동·청소년성착취물 처벌 강화	, 
818	: 「벤처투자 촉진에 관한 법률」 전면 시행
749, 735, 734, 738, 737:  어린이 안전 (20년 상반기)
752 : 아동학대 : (21년 상반기 아동학대 7번토픽)
830 : 국내 외국인 투자기업 사내유보금 재투자시 외국인투자로 인정	(2019하반기 4번)
743 : 성범죄


#2-0:
490 : 사각지대 없는 위기청소년 보호·지원체계 구축	
544 : 아동·청소년 대상 성범죄 신고의무 및 취업제한 확대	
545 : 성범죄성희롱·성폭력 근절 종합지원센터 운영	
481 : 창업중소기업 세액감면 대상 업종 확대 (19년도 상반기 10번 - 청년)
546 : 성범죄자 신상정보 모바일 전자고지 시행


#1-1:

'''

'\n3-1 : \n1264 : \n1216 : 2020년 하반기 9번 - 중소기업 : 기술 보유 혁싱 중소기업 등록관련 = 항상 등장하는 기업 기술 관련 토픽\n1194 : 교정시설 코로나(2020년 하반기 구치소, 4,7,10)\n1207 : 아동 성 착취 (21년 상반기 아동학대 7번토픽)\n1301 : 백신 개발 관련 정책\n1211 : 디지털 성범죄 :\n1208 : 성범죄자 신상정보 모바일 고지 확대\t\n\n'

In [ ]:
years = [2018,2019,2020,2021]
halfs = ['상반기','하반기']
yy = years[3]
hh = halfs[1]
temp = data[(data['year']==yy) & (data['period']==hh)]
temp.sort_values(by = 'score', ascending = False)



,분야,번호,정책명,정책내용,year,period,nouns,score
1264,산업/중소기업/에너지,4,통신재난 예방·대응을 위한 관리·점검체계 개선,통신재난 예방 대응을 위한 관리 점검체계 개선 네트워크안전기획과 ...,2021,하반기,통신 재난 예방 대응 관리 점검 체계 개선 네트워크 안전 기획 분야 산업 중소기업 ...,36
1194,행정/안전/질서,3,법무부 코로나19 교정시설 긴급대응단 운영 실시,법무부 코로나 교정시설 긴급대응단 운영 실시 코로나 긴급대응단 ...,2021,하반기,법무부 코로나 교정 시설 긴급 대응 운영 실시 코로나 긴급 대응 분야 행정 안전 질...,30
1213,행정/안전/질서,22,공공기관 규정을 국가법령정보센터에서 통합하여 제공,공공기관 규정을 국가법령정보센터에서 통합하여 제공 법제정보담당관실 ...,2021,하반기,공공기관 규정 국가 법령 정보 센터 통합 제공 정보 담당 관실 분야 행정 안전 질서...,28
1226,행정/안전/질서,35,통신판매업 신고증 온라인 발급 시행,통신판매업 신고증 온라인 발급 시행 전자거래과 정부 ...,2021,하반기,통신 판매업 고증 온라인 발급 시행 전자거래 정부 운영 분야 행정 안전 질서 대상 ...,25
1207,행정/안전/질서,16,"온라인에서의 아동·청소년 성적 착취 목적의 그루밍 처벌, 아동·청소년 대상 디지털 ...",온라인에서의 아동 청소년 성적 착취 목적의 그루밍 처벌 아동 청소년 대상 디지털 ...,2021,하반기,온라인 아동 청소년 성적 착취 목적 그루밍 처벌 아동 청소년 대상 디지털 성범죄 분...,24
1185,국방/병무,13,예술·체육요원 특기활용 공익복무(봉사활동) 부실자 제재 강화,예술 체육요원 특기활용 공익복무 봉사활동 부실자 제재 강화 사회복무정책과 ...,2021,하반기,예술 체육 요원 특기 활용 공익 복무 봉사활동 부실 제재 강화 사회 복무 정책 분야...,23
1172,교육/보육/가족,9,학교 밖 청소년 자동 정보연계 시행,학교 밖 청소년 자동 정보연계 시행 학교밖청소년지원과 분...,2021,하반기,학교 청소년 자동 정보 시행 학교 청소년 지원 분야 교육 보육 가족 대상 청소년 관...,23
1298,보건/복지/고용,21,산재보험 진료비 본인부담금 확인제도 시행,산재보험 진료비 본인부담금 확인제도 시행 산재보상정책과 ...,2021,하반기,산재 보험 진료 인부 확인 제도 시행 산재 보상 정책 분야 보건 복지 고용 대상 기...,20
1212,행정/안전/질서,21,과징금 납부기한 연기 및 분할 납부 확대,과징금 납부기한 연기 및 분할 납부 확대 행정법제 혁신 추진단 총괄팀 ...,2021,하반기,과징금 납부 기한 연기 분할 납부 확대 행정법 혁신 추진단 총괄 분야 행정 안전 질...,20
1152,금융/재정/조세,8,부가가치세 간이과세제도 개편,부가가치세 간이과세제도 개편 부가가치세과 분야 금융 재정...,2021,하반기,부가가치세 간이 과세 제도 개편 부가가치세 분야 금융 재정 조세 대상 사업자 관련 ...,19


In [ ]:
fin.iloc[6].top_keywords

['수산',
 '식품',
 '개발',
 '해양',
 '보건',
 '의료',
 '주택',
 '구축',
 '코로나',
 '검사',
 '행정',
 '활용',
 '제출',
 '바이오',
 '농림',
 '급식',
 '체계',
 '복무',
 '보육',
 '질서',
 '예방',
 '가입',
 '위험',
 '건강',
 '포함',
 '중소기업',
 '자료',
 '장애인',
 '심사',
 '미만',
 '거래',
 '분석',
 '활성화',
 '온라인',
 '지정',
 '문화',
 '농촌',
 '학생',
 '문화재',
 '발급',
 '규정',
 '항만',
 '투자',
 '시간',
 '선박',
 '실시',
 '학교',
 '대학',
 '발생',
 '전국']

# LDA

In [ ]:
%ls

 정책_2018_01.csv   정책_2020_02.csv          '정책 tfidf_0524.csv'
 정책_2018_02.csv   정책_2021_01.csv          '정책 tfidf.csv'
 정책_2019_01.csv   정책_2021_02.csv           정책_tfidf.csv
 정책_2019_02.csv   정책LDA_2018_상반기.html  '정책 tfidf.gsheet'
 정책_2020_01.csv   정책_preprocessed.csv


In [ ]:
data.head()

,분야,번호,정책명,정책내용,year,period,nouns
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,법정 최고금리 연 로 인하 법무부 상사법무과 금...,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축 법무부 상사법무과 분야 금...,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입 방송통신위원회 개인정보보호윤리과 ...,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,외국인관광객 숙박요금 부가가치세 환급 문화체육관광부 관광산업정책과 ...,2018,상반기,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점 조달청 기술서비스총괄과 ...,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...


In [ ]:
def get_corpus(yy, hh):
    corpus = []
    temp = data[(data['year']==yy) & (data['period']==hh)]
    for i in temp['nouns']:
        tmp = i.split()
        tmp = [j for j in tmp if j not in stopwords]
        corpus.append(tmp)
    return corpus

In [ ]:
years = [2018,2019,2020,2021]
halfs = ['상반기','하반기']

In [ ]:
for yy in years:
  for hh in halfs:
    corpus = get_corpus(yy, hh)
    dictionary = corpora.Dictionary(corpus)
    # 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
    dictionary.filter_extremes(no_below=2, no_above=0.5)
    #dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
    corpus = [dictionary.doc2bow(text) for text in corpus]
    num_topics = 5
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None

    temp = dictionary[0]
    id2word = dictionary.id2token

    model = LdaModel(
      corpus=corpus,
      id2word=id2word,
      chunksize=chunksize,
      alpha='auto',
      eta='auto',
      iterations=iterations,
      num_topics=num_topics,
      passes=passes,
      eval_every=eval_every
    )

    top_topics = model.top_topics(corpus) #, num_words=20)

    # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
    avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
    print('Average topic coherence: %.4f.' % avg_topic_coherence)

    from pprint import pprint
    pprint(top_topics)

    lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
    pyLDAvis.save_html(lda_visualization, '정책LDA05_{}_{}.html'.format(yy,hh))


2022-05-26 08:37:10,110 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-26 08:37:10,120 : INFO : built Dictionary(1292 unique tokens: ['가계부채', '갱신', '거래', '경감', '경제']...) from 83 documents (total 6743 corpus positions)
2022-05-26 08:37:10,126 : INFO : discarding 766 tokens: [('가계부채', 1), ('갱신', 1), ('국무회의', 1), ('금융기관', 1), ('금융위', 1), ('금전', 1), ('대부', 1), ('마당', 1), ('법정', 1), ('부업', 1)]...
2022-05-26 08:37:10,128 : INFO : keeping 526 tokens which were in no less than 2 and no more than 41 (=50.0%) documents
2022-05-26 08:37:10,133 : INFO : resulting dictionary: Dictionary(526 unique tokens: ['거래', '경감', '경제', '계약', '계층']...)
2022-05-26 08:37:10,141 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2022-05-26 08:37:10,143 : INFO : using serial LDA version on this node
2022-05-26 08:37:10,148 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 83 documents, updating model once every 83 document

Average topic coherence: -13.8180.
[([(0.019126602, '조정'),
   (0.017824164, '범위'),
   (0.016522048, '양도소득세'),
   (0.016285555, '세액'),
   (0.016229102, '기한'),
   (0.014514544, '가격'),
   (0.013962028, '개월'),
   (0.013652805, '주식'),
   (0.013496373, '주택'),
   (0.013298868, '단축'),
   (0.013192853, '한도'),
   (0.012944708, '일반'),
   (0.01237824, '체납'),
   (0.0117663685, '연장'),
   (0.011547612, '공개'),
   (0.01107238, '수입'),
   (0.010712676, '국회'),
   (0.010707853, '관세'),
   (0.0106630735, '양도'),
   (0.010037583, '처분')],
  -10.397110534512093),
 ([(0.023843123, '대출'),
   (0.018904623, '비율'),
   (0.01812482, '중견'),
   (0.014773045, '주택'),
   (0.014454659, '관세'),
   (0.014010516, '기한'),
   (0.013230859, '특수'),
   (0.012470835, '경감'),
   (0.012068766, '법인'),
   (0.012042206, '병원'),
   (0.011099044, '한도'),
   (0.01103252, '중소기업'),
   (0.011008483, '주거'),
   (0.010991724, '경력'),
   (0.01092415, '금리'),
   (0.010847284, '전세'),
   (0.010798098, '거래'),
   (0.010404756, '청년'),
   (0.010400452, '억원'),
  

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
2022-05-26 08:37:14,176 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-26 08:37:14,180 : INFO : built Dictionary(924 unique tokens: ['가구', '건설', '계층', '교통', '국토']...) from 30 documents (total 3023 corpus positions)
2022-05-26 08:37:14,192 : INFO : discarding 671 tokens: [('가구', 1), ('건설', 1), ('급여', 1), ('달리', 1), ('마이', 1), ('보수', 1), ('부양', 1), ('비수', 1), ('빈곤층', 1), ('사비', 1)]...
2022-05-26 08:37:14,195 : INFO : keeping 253 tokens which were in no less than 2 and no more than 15 (=50.0%) documents
2022-05-26 08:37:14,198 : INFO : resulting dictionary: Dictionary(253 unique tokens: ['계층', '교통', '국토', '그간', '기능']...)
2022-05-26 08:37:14,206 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0

Average topic coherence: -13.2930.
[([(0.06379129, '물품'),
   (0.04290345, '책임'),
   (0.03411927, '종합'),
   (0.023433551, '방위사업청'),
   (0.022301842, '계약'),
   (0.021409726, '제출'),
   (0.020063166, '업체'),
   (0.019749762, '특수'),
   (0.019054621, '가능'),
   (0.018838456, '완화'),
   (0.0182355, '방산'),
   (0.017533338, '조달청'),
   (0.015371233, '사유'),
   (0.014131352, '제한'),
   (0.012865876, '대가'),
   (0.012825353, '방위'),
   (0.012373782, '자격'),
   (0.012136475, '근거'),
   (0.011914256, '대해'),
   (0.011914227, '자료')],
  -8.565947137448475),
 ([(0.06926656, '입영'),
   (0.051055364, '연기'),
   (0.03377908, '보장'),
   (0.030333962, '사유'),
   (0.026756046, '일자'),
   (0.021764632, '제한'),
   (0.019178368, '병무청'),
   (0.01907257, '근무'),
   (0.018614667, '선발'),
   (0.017352777, '병역의무'),
   (0.015918221, '특수'),
   (0.015805252, '업무'),
   (0.015435788, '군인'),
   (0.015283833, '생활'),
   (0.015116803, '상담'),
   (0.014993193, '개월'),
   (0.014588043, '휴가'),
   (0.01458804, '활동'),
   (0.014566739, '국방부'),
   (0.

2022-05-26 08:37:15,950 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-26 08:37:15,966 : INFO : built Dictionary(1671 unique tokens: ['계층', '공급', '공인', '금리', '기업은행']...) from 126 documents (total 10098 corpus positions)
2022-05-26 08:37:15,974 : INFO : discarding 1008 tokens: [('기업은행', 1), ('미래', 1), ('바탕', 1), ('상환', 1), ('수익', 1), ('자동', 1), ('조달', 1), ('초반', 1), ('출금', 1), ('경유', 1)]...
2022-05-26 08:37:15,976 : INFO : keeping 663 tokens which were in no less than 2 and no more than 63 (=50.0%) documents
2022-05-26 08:37:15,983 : INFO : resulting dictionary: Dictionary(663 unique tokens: ['계층', '공급', '공인', '금리', '대금']...)
2022-05-26 08:37:15,996 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2022-05-26 08:37:15,999 : INFO : using serial LDA version on this node
2022-05-26 08:37:16,004 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 126 documents, updating model once every 126 document

...

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
2022-05-26 08:37:31,015 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-26 08:37:31,046 : INFO : built Dictionary(3662 unique tokens: ['각종', '건물', '경제', '극복', '기본']...) from 296 documents (total 26152 corpus positions)
2022-05-26 08:37:31,058 : INFO : discarding 1942 tokens: [('열거', 1), ('포스트', 1), ('공시가격', 1), ('성립', 1), ('정시', 1), ('양도소득', 1), ('결손', 1), ('리스크', 1), ('이익', 1), ('장기간', 1)]...
2022-05-26 08:37:31,061 : INFO : keeping 1720 tokens which were in no less than 2 and no more than 148 (=50.0%) documents
2022-05-26 08:37:31,067 : INFO : resulting dictionary: Dictionary(1720 unique tokens: ['각종', '건물', '경제', '극복', '기본']...)
2022-05-26 08:37:31,105 : INFO : using autotuned alpha, starting with [0.2, 0.2

Average topic coherence: -10.2601.
[([(0.012857351, '해양'),
   (0.0121656, '의료'),
   (0.011398194, '수산'),
   (0.011252221, '개발'),
   (0.009167864, '식품'),
   (0.008065764, '구축'),
   (0.0064407866, '활용'),
   (0.0054827896, '농림'),
   (0.005431949, '분석'),
   (0.00542487, '체계'),
   (0.005367061, '질병'),
   (0.005362345, '연구'),
   (0.00528584, '신규'),
   (0.004940867, '바이오'),
   (0.0049359817, '기반'),
   (0.0046784976, '법령'),
   (0.004436451, '안정'),
   (0.0042537544, '대응'),
   (0.0042387224, '최대'),
   (0.0041900384, '인력')],
  -4.395113231378311),
 ([(0.019223496, '주택'),
   (0.009297095, '선박'),
   (0.008133341, '항만'),
   (0.007560483, '식품'),
   (0.007133834, '급식'),
   (0.006150567, '시장'),
   (0.006091254, '치유'),
   (0.005912054, '세율'),
   (0.005759478, '문화'),
   (0.0053900173, '검사'),
   (0.005191572, '종합'),
   (0.005127749, '포함'),
   (0.0050877933, '억원'),
   (0.0049347533, '의료'),
   (0.0048365598, '조정'),
   (0.0048196395, '세대'),
   (0.004751286, '수산'),
   (0.004739915, '보유'),
   (0.0047292397, '해

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
2022-05-26 08:37:36,468 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-26 08:37:36,489 : INFO : built Dictionary(2788 unique tokens: ['가산', '간이', '경감', '근로', '금액']...) from 166 documents (total 16577 corpus positions)
2022-05-26 08:37:36,498 : INFO : discarding 1504 tokens: [('반기', 1), ('불분명', 1), ('원천', 1), ('일용', 1), ('소상', 1), ('수의', 1), ('시화', 1), ('유연성', 1), ('발행', 1), ('개산', 1)]...
2022-05-26 08:37:36,502 : INFO : keeping 1284 tokens which were in no less than 2 and no more than 83 (=50.0%) documents
2022-05-26 08:37:36,507 : INFO : resulting dictionary: Dictionary(1284 unique tokens: ['가산', '간이', '경감', '근로', '금액']...)
2022-05-26 08:37:36,530 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2,

Average topic coherence: -12.6964.
[([(0.009865955, '계약'),
   (0.008630949, '산재'),
   (0.0070595522, '업체'),
   (0.0064812256, '법령'),
   (0.0063939723, '사업자'),
   (0.0062150904, '활동'),
   (0.0061989618, '규정'),
   (0.0057365466, '확인'),
   (0.0057103485, '참여'),
   (0.005624738, '예술'),
   (0.0055342917, '해양'),
   (0.0055226223, '상품'),
   (0.0052681966, '센터'),
   (0.0052631204, '직접'),
   (0.005135878, '실시'),
   (0.004953605, '급여'),
   (0.0049014445, '방문'),
   (0.0048671593, '부대'),
   (0.0047981963, '종사'),
   (0.0047590276, '매장')],
  -10.6266540517843),
 ([(0.013084442, '대응'),
   (0.010778414, '통신'),
   (0.008201971, '피해'),
   (0.007684513, '정보보호'),
   (0.007256007, '실습'),
   (0.007162409, '재난'),
   (0.00648815, '조치'),
   (0.0061347787, '가입'),
   (0.006030113, '개발'),
   (0.0060014725, '체계'),
   (0.0057668188, '공무원'),
   (0.005655019, '개인'),
   (0.00555031, '센터'),
   (0.005375093, '예방'),
   (0.0052658715, '부여'),
   (0.0051411875, '부품'),
   (0.0051374263, '허용'),
   (0.0051311255, '설치'),
   (0.

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
corpus_201801 = get_corpus(2021, '상반기')

In [ ]:
corpus_201801

[['통합',
  '투자',
  '세액',
  '특례',
  '코로나',
  '피해',
  '조기',
  '극복',
  '경제',
  '수준',
  '상이',
  '각종',
  '투자',
  '세액',
  '통합',
  '설계',
  '통합',
  '투자',
  '세액',
  '세제',
  '자산',
  '열거',
  '특정',
  '중심',
  '모든',
  '일반',
  '유형',
  '자산',
  '대폭',
  '토지',
  '건물',
  '차량',
  '비품',
  '일부',
  '자산',
  '제외',
  '투자',
  '증가',
  '부여',
  '투자',
  '증가',
  '인센티브',
  '성장',
  '투자',
  '투자',
  '기본',
  '소득세',
  '법인세',
  '투자',
  '대해',
  '통합',
  '투자',
  '세액',
  '특정',
  '투자',
  '세액',
  '선택',
  '허용',
  '코로나',
  '피해',
  '극복',
  '포스트',
  '코로나',
  '대비',
  '특정',
  '투자',
  '세액',
  '중소기업',
  '투자',
  '세액',
  '통합',
  '설계',
  '세제',
  '자산',
  '투자',
  '증가',
  '센티',
  '부여',
  '성장',
  '투자',
  '우대',
  '소득세',
  '법인세',
  '투자',
  '대해',
  '통합',
  '투자',
  '세액',
  '특정',
  '투자',
  '세액',
  '선택',
  '허용'],
 ['주택',
  '종합',
  '부동산',
  '세율',
  '인상',
  '재산세',
  '제과',
  '주택',
  '시장',
  '안정',
  '주택',
  '종합',
  '부동산',
  '세제',
  '주택',
  '종합',
  '부동산',
  '세율',
  '인상',
  '법인',
  '대해',
  '개인',
  '최고',
  '세율',
  '세율',
  '조정',
  '주택',
  '자의',
  '상한',
  '인상',

In [ ]:
len(corpus_201801)

296

In [ ]:
dictionary = corpora.Dictionary(corpus_201801)
# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in corpus_201801]

2022-05-25 13:24:11,798 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 13:24:11,840 : INFO : built Dictionary(3662 unique tokens: ['각종', '건물', '경제', '극복', '기본']...) from 296 documents (total 26152 corpus positions)


In [ ]:
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 13:24:11,886 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2022-05-25 13:24:11,890 : INFO : using serial LDA version on this node
2022-05-25 13:24:11,900 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 296 documents, updating model once every 296 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 13:24:11,904 : INFO : PROGRESS: pass 0, at document #296/296
2022-05-25 13:24:12,773 : INFO : optimized alpha [0.064133674, 0.063734934, 0.0647511, 0.06057595, 0.058376435, 0.056178242, 0.06349667, 0.060085744, 0.05799323, 0.06682752]
2022-05-25 13:24:12,783 : INFO : topic #5 (0.056): 0.012*"문화" + 0.011*"농촌" + 0.008*"생활" + 0.007*"발급" + 0.007*"바닷가" + 0.006*"심의" + 0.006*"허가" + 0.006*"연구" + 0.006*"가구" + 0.006*"증명서"
2022-05-25 13:24:12,786 : INFO : topic #8 (0.058): 0.009*"주류" + 0.009*"국내" + 0.009*"납부" + 0.007*"보증" + 0

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -11.9408.
[([(0.01476365, '투자'),
   (0.010494858, '거래'),
   (0.008312325, '확인'),
   (0.007459832, '구축'),
   (0.0071057277, '에너지'),
   (0.0071049123, '온라인'),
   (0.0070940023, '활용'),
   (0.0060840594, '개발'),
   (0.0060018697, '농업'),
   (0.00592293, '조치'),
   (0.0058963303, '벤처기업'),
   (0.0058798604, '자산'),
   (0.0057820086, '유통'),
   (0.0055941595, '중소기업'),
   (0.0053151394, '방지'),
   (0.005311223, '활성화'),
   (0.005260177, '분석'),
   (0.0050471453, '사업자'),
   (0.004941394, '디지털'),
   (0.0048250533, '소비자')],
  -7.653502812105706),
 ([(0.009997896, '복무'),
   (0.009038922, '장애인'),
   (0.0067117685, '정보공개'),
   (0.00630907, '활용'),
   (0.0061490564, '행정'),
   (0.0059934477, '데이터'),
   (0.0055697355, '통합'),
   (0.0053028907, '평생교육'),
   (0.0051145772, '보건'),
   (0.005034412, '체계'),
   (0.0049411925, '활동'),
   (0.0047129216, '실시'),
   (0.004693307, '시험'),
   (0.0045236233, '대학'),
   (0.0045047044, '검사'),
   (0.004479741, '단계'),
   (0.004473147, '장애'),
   (0.004412593, '

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, '정책LDA_2018_상반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
